<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# NLP - Bot basado en reglas con Tensorflow
Este ejemplo consiste en armar BOT simple basado en una red neuronal con Tensorflow

v1.0

In [ ]:
import json
import string
import random 
import numpy as np

import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

import pickle

# Recolectar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline1.png" width="1000" align="middle">

In [ ]:
# Dataset en formato JSON que representa las posibles preguntas (patterns)
# y las posibles respuestas por categoría (tag)
data = {"intents": [
             {"tag": "bienvenida",
              "patterns": ["Hola", "¿Cómo estás?", "¿Qué tal?"],
              "responses": ["Hola!", "Hola, ¿Cómo estás?"],
             },
             {"tag": "nombre",
              "patterns": ["¿Cúal es tu nombre?", "¿Quién sos?"],
              "responses": ["Mi nombre es MarvelBOT", "Yo soy MarvelBOT"]
             },
            {"tag": "contacto",
              "patterns": ["contacto", "número de contacto", "número de teléfono", "número de whatsapp", "whatsapp"],
              "responses": ["Podes contactarnos al siguiente número +54-9-11-2154-4777", "Contactonos al whatsapp número +54-9-11-2154-4777"]
             },
            {"tag": "envios",
              "patterns": ["¿Realizan envios?", "¿Cómo me llega el paquete?"],
              "responses": ["Los envios se realizan por correo, lo enviaremos a la dirección que registraste en la página"]
             },
            {"tag": "precios",
              "patterns": ["precio", "Me podrás pasar los precios", "¿Cuánto vale?", "¿Cuánto sale?"],
              "responses": ["En el catálogo podrás encontrar los precios de todos nuestros productos en stock"]
             },
            {"tag": "pagos",
              "patterns": ["medios de pago", "tarjeta de crédito", "tarjetas", "cuotas"],
              "responses": ["Contactanos al whatsapp número +54-9-11-2154-4777 para conocer los beneficios y formas de pago vigentes"]
             },
            {"tag": "stock",
              "patterns": ["Esto está disponible", "¿Tenes stock?", "¿Hay stock?"],
              "responses": ["Los productos publicados están en stock"]
             },
            {"tag": "agradecimientos",
              "patterns": [ "Muchas gracias", "Gracias"],
              "responses": ["Por nada!, cualquier otra consulta podes escribirnos"]
             },
             {"tag": "despedida",
              "patterns": [ "Chau", "Hasta luego!"],
              "responses": ["Hasta luego!", "Hablamos luego!"]
             }
]}

# Procesar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline2.png" width="1000" align="middle">

### Herramientas de preprocesamiento de datos
Entre las tareas de procesamiento de texto en español se implementa:
- Quitar números
- Quitar símbolos de puntuación
- Quitar caracteres acentuados

In [ ]:
import re
import string

# El preprocesamento en castellano requiere más trabajo

def preprocess_clean_text(text):
    # pasar a minúsculas
    text = text.lower()
    # quitar números
    pattern = r'[0-9\n]'
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntiación
    text = ''.join([c for c in text if c not in (string.punctuation+"¡"+"¿")])
    # quitar caracteres con acento
    text = re.sub(r'[àáâä]', "a", text)
    text = re.sub(r'[éèêë]', "e", text)
    text = re.sub(r'[íìîï]', "i", text)
    text = re.sub(r'[òóôö]', "o", text)
    text = re.sub(r'[úùûü]', "u", text)
    return text

In [ ]:
string.punctuation + "¡" + "¿"

In [ ]:
preprocess_clean_text("¿cómo5!")

### Lematizacion

In [ ]:
import os
import gdown
if os.access('lematizacion-es.pickle', os.F_OK) is False:
    !curl -L -o 'lematizacion-es.zip' 'https://drive.google.com/u/0/uc?id=16leuM9PuFXAkmw34XeQy-84h8WGAYxJw&export=download&confirm=t'
    !unzip -q lematizacion-es.zip
else:
    print("El archivo ya se encuentra descargado")

In [ ]:
with open("lematizacion-es.pickle",'rb') as fi:
    lemma_lookupTable = pickle.load(fi)

In [ ]:
# Ejemplo del funcionamiento de lematización
palabras_ensayo = ["estar", "estoy", "estás", "está", "estamos" ,"estais", "estan", "estaremos", "estuvieron"]
for palabra in palabras_ensayo:
    print(f"{palabra} -> {lemma_lookupTable.get(palabra)}")

### Preprocesado del dataset

In [ ]:
words = []
classes = []
doc_X = []
doc_y = []
# Tokenizar cada "pattern" y agregar cada palabra al vocabulario (vocabulary)
# Los tokens que se toman de cada pattern se agrega a doc_X
# Cada tag se agrega a doc_y
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        # trasformar el patron a tokens
        tokens = preprocess_clean_text(pattern).split(" ")
        # lematizar los tokens
        lemma_words = []
        for token in tokens:
            lemma = lemma_lookupTable.get(token)
            if lemma is not None:
                lemma_words.append(lemma)
            else:
                print("UNK:", token)
        
        if not lemma_words:
            continue
        
        words += lemma_words
        doc_X.append(pattern)
        doc_y.append(intent["tag"])
    
    # Agregar el tag a las clases
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# Elminar duplicados con "set" y ordenar el vocubulario y las clases por orden alfabético
vocab = sorted(set(words))
classes = sorted(set(classes))
len(vocab)

# Explorar datos
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline3.png" width="1000" align="middle">

In [ ]:
print("vocab:", vocab)
print("classes:", classes)
print("doc_X:", doc_X)
print("doc_y:", doc_y)

In [ ]:
doc_y_encoded = [classes.index(label) for label in doc_y]
doc_y_encoded

# Entrenar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline4.png" width="1000" align="middle">

In [ ]:
X_train = []
for doc in doc_X:
    # Transformar la pregunta (input) en tokens y lematizar
    lemma_words = []
    tokens = preprocess_clean_text(doc).split(" ")
    for token in tokens:
        lemma = lemma_lookupTable.get(token)
        if lemma is not None:
            lemma_words.append(lemma)

    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    bow = []
    for word in vocab:
        bow.append(1) if word in lemma_words else bow.append(0)
    
    print("X:", bow)
    X_train.append(bow)

X_train = np.array(X_train)

In [ ]:
X_train.shape

In [ ]:
y_train = tf.keras.utils.to_categorical(doc_y_encoded)
y_train[:4]

In [ ]:
input_shape = X_train.shape[1]
output_shape = y_train.shape[1]
print("input:", input_shape, "output:", output_shape)

In [ ]:
# Entrenamiento del modelo DNN
# - Modelo secuencial
# - Con regularización
# - softmax y optimizador Adam
model = Sequential()
model.add(Dense(128, activation="relu", input_shape=(input_shape,)))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(output_shape, activation = "softmax"))

model.compile(loss='categorical_crossentropy',
              optimizer="Adam",
              metrics=["accuracy"])
print(model.summary())

In [ ]:
hist = model.fit(x=X_train, y=y_train, epochs=200, verbose=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Entrenamiento
epoch_count = range(1, len(hist.history['accuracy']) + 1)
sns.lineplot(x=epoch_count,  y=hist.history['accuracy'], label='train')
plt.show()

# Utilizar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline6.png" width="1000" align="middle">

In [ ]:
responses = [[""]] * len(classes)
for intent in data["intents"]:
    responses[classes.index(intent["tag"])] = intent["responses"]

responses

In [ ]:
for i in range(5):
    message = input("")

    # preprocesamiento + lematizacion
    # ------------------------------------------
    # Transformar la pregunta (input) en tokens y lematizar
    lemma_words = []
    tokens = preprocess_clean_text(message).split(" ")
    for token in tokens:
        lemma = lemma_lookupTable.get(token)
        if lemma is not None:
            lemma_words.append(lemma)

    # Transformar los tokens en "Bag of words" (arrays de 1 y 0)
    bow = []
    for word in vocab:
        bow.append(1) if word in lemma_words else bow.append(0)
    # ------------------------------------------

    probs = model.predict([bow])
    score = probs.max()
    if score > 0.4:  # threshold 0.4        
        index = probs.argmax(axis=1)[0]
        result = random.choice(responses[index])
        print(f"[{score:.2f}]: {result}")
    else:
        print(f"[{score:.2f}] Perdon, no comprendo la pregunta.")

# Descargar modelo
<img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/Pipeline7.png" width="1000" align="middle">

En el repositorio de clase ya se encuentran descargados estos archivos dentro de una carpeta llamada "model". El código a continuación es un ejemplo de como se descargaron por si usted quiere editar el bot y descargar los archivos necesarios para ejecutarlo / deployarlo.

In [ ]:
# Exportar los datos importants (vocabulario, clases, el bot y el dataset utilizado)
import pickle
import json

pickle.dump(vocab, open('vocab.pkl','wb'))
pickle.dump(responses, open('responses.pkl','wb'))
model.save('bot.h5')

In [ ]:
# Comprimir todos los datos necesarios
!zip -r bot_data.zip bot.h5 vocab.pkl responses.pkl lematizacion-es.pickle

In [ ]:
from google.colab import files
files.download('bot_data.zip') 